# Move speed experiments

## Setup

In [1]:
import lysis
import math
# import numpy as np
import cupy as np
from functools import partial

In [2]:
e = lysis.util.Experiment(r'../../data', experiment_code='2022-12-27-1100')
p = {'total_time': 1}
e.initialize_macro_param(p)
macro = lysis.MacroscaleRun(e)

In [3]:
macro.run()

100%|█████████████████████████████████████████████████████████████████████████████| 2920/2920 [00:08<00:00, 344.46it/s]

Total binds: 10,552
Timesteps with binds: 2,770
Total moves: 21,417,478


In [4]:
rng = np.random.default_rng(seed=489489496)

In [5]:
def generate_locations():
    locationx = rng.integers(e.macro_params.rows-1,
                             size=e.macro_params.total_molecules,
                             dtype=np.short)
    locationy = rng.integers(e.macro_params.full_row,
                             size=e.macro_params.total_molecules,
                             dtype=np.short)
    neighbor_k = rng.integers(8,
                              size=e.macro_params.total_molecules,
                              dtype=np.short)
    location = np.concatenate((locationx, locationy)).reshape((2, -1))
    location_flat = np.ravel_multi_index(tuple(location), (e.macro_params.rows, e.macro_params.full_row))
    return locationx, locationy, location, neighbor_k, location_flat

In [20]:
neighbors_i = np.array(macro.neighbors_i)
neighbors_j = np.array(macro.neighbors_j)
neighbors = np.empty(macro.neighbors_i.shape + (2,), dtype=np.short)
for i, j, k in np.ndindex(macro.neighbors_i.shape):
    neighbors[i, j, k, 0] = neighbors_i[i, j, k]
    neighbors[i, j, k, 1] = neighbors_j[i, j, k]

In [17]:
neighbors_flat = np.empty((e.macro_params.rows * e.macro_params.full_row, 8, 2), dtype=np.ushort)
for i, j, k in np.ndindex(macro.neighbors_i.shape):
    neighbors_flat[np.ravel_multi_index((i, j,), macro.neighbors_i.shape[:2]), k, 0] = neighbors_i[i, j, k]
    neighbors_flat[np.ravel_multi_index((i, j,), macro.neighbors_i.shape[:2]), k, 1] = neighbors_j[i, j, k]

In [35]:
neighbors_flat_flat = np.empty((e.macro_params.rows * e.macro_params.full_row, 8), dtype=np.ushort)
for i, j, k in np.ndindex(macro.neighbors_i.shape):
    neighbors_flat_flat[np.ravel_multi_index((i, j,), macro.neighbors_i.shape[:2]), k] = np.ravel_multi_index((neighbors_i[i, j, k], neighbors_j[i, j, k],), macro.neighbors_i.shape[:2])

AttributeError: 'int' object has no attribute 'shape'

In [37]:
neighbors_ijdk = np.empty((macro.neighbors_i.shape[:2] + (2, 8,)), dtype=np.short)
for i, j, k in np.ndindex(macro.neighbors_i.shape):
    neighbors_ijdk[i, j, 0, k] = neighbors_i[i, j, k]
    neighbors_ijdk[i, j, 1, k] = neighbors_j[i, j, k]

In [38]:
neighbors_kijd = np.empty(((8,) + macro.neighbors_i.shape[:2] + (2,)), dtype=np.short)
for i, j, k in np.ndindex(macro.neighbors_i.shape):
    neighbors_kijd[k, i, j, 0] = neighbors_i[i, j, k]
    neighbors_kijd[k, i, j, 1] = neighbors_j[i, j, k]

In [39]:
neighbors_dijk = np.empty(((2,) + macro.neighbors_i.shape[:2] + (8,)), dtype=np.short)
for i, j, k in np.ndindex(macro.neighbors_i.shape):
    neighbors_dijk[0, i, j, k] = neighbors_i[i, j, k]
    neighbors_dijk[1, i, j, k] = neighbors_j[i, j, k]

In [40]:
neighbors_dkij = np.empty(((2, 8,) + macro.neighbors_i.shape[:2]), dtype=np.short)
for i, j, k in np.ndindex(macro.neighbors_i.shape):
    neighbors_dkij[0, k, i, j] = neighbors_i[i, j, k]
    neighbors_dkij[1, k, i, j] = neighbors_j[i, j, k]

In [41]:
neighbors_kdij = np.empty(((8, 2,) + macro.neighbors_i.shape[:2]), dtype=np.short)
for i, j, k in np.ndindex(macro.neighbors_i.shape):
    neighbors_kdij[k, 0, i, j] = neighbors_i[i, j, k]
    neighbors_kdij[k, 1, i, j] = neighbors_j[i, j, k]

In [ ]:
neighbor_dict = {}
for i in range(e.macro_params.rows):
    for j in range(e.macro_params.cols):
        for k in range(8):
            

In [21]:
fiber_status = np.array(macro.fiber_status)

In [22]:
locationx, locationy, location, neighbor_k, location_flat = generate_locations()

## Multiple location arrays

### Multiple neighbor arrays

In [23]:
type(neighbors_i)#[locationx, locationy, neighbor_k]

cupy.ndarray

In [24]:
%timeit temp = neighbors_i[locationx, locationy, neighbor_k]; newloc_j = neighbors_j[locationx, locationy, neighbor_k]; newloc_i = temp

287 µs ± 93.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Single neighbor array

In [25]:
%timeit newloc = neighbors[locationx, locationy, neighbor_k]; newloc_i = newloc[:,0]; newloc_j = newloc[:,1]

138 µs ± 22 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Fiber Status

In [26]:
%timeit m_fiber_status = fiber_status[locationx, locationy]

157 µs ± 21.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Single location array

## Multiple neighbor arrays

In [49]:
%timeit newloc = np.array([neighbors_i[location[0], location[1], neighbor_k], neighbors_j[location[0], location[1], neighbor_k]])

368 µs ± 40.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
%timeit newloc = np.array([neighbors_i[tuple(location) + (neighbor_k,)], neighbors_j[tuple(location) + (neighbor_k,)]])

360 µs ± 4.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [31]:
%timeit newloc = np.concatenate((neighbors_i[tuple(location) + (neighbor_k,)], neighbors_j[tuple(location) + (neighbor_k,)])).reshape((2, -1))

364 µs ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Single neighbor array

In [32]:
%timeit newloc = neighbors[location[0], location[1], neighbor_k].T

155 µs ± 8.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [33]:
%timeit newloc = neighbors[tuple(location) + (neighbor_k,)].T

150 µs ± 4.8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [48]:
%timeit newloc = neighbors_ijdk[location[0], location[1], :, neighbor_k].T

153 µs ± 5.76 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [43]:
%timeit newloc = neighbors_kijd[neighbor_k, location[0], location[1]]

157 µs ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [44]:
%timeit newloc = neighbors_kijd[(neighbor_k,) + tuple(location)].T

157 µs ± 4.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [45]:
%timeit newloc = neighbors_dijk[:, location[0], location[1], neighbor_k]

154 µs ± 6.52 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [46]:
%timeit newloc = neighbors_dkij[:, neighbor_k, location[0], location[1]]

148 µs ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [47]:
%timeit newloc = neighbors_kdij[neighbor_k, :, location[0], location[1]]

153 µs ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [31]:
%timeit newloc = neighbors_flat_flat[np.ravel_multi_index(tuple(location), (e.macro_params.rows, e.macro_params.full_row)), neighbor_k]

445 µs ± 8.58 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
%timeit newloc = neighbors_flat_flat[location_flat, neighbor_k]

248 µs ± 3.43 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Fiber status

In [36]:
%timeit m_fiber_status = fiber_status[location[0], location[1]]

142 µs ± 20 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
locationx, locationy, location, neighbor_k = generate_locations()
%timeit m_fiber_status = fiber_status[tuple(location)]

257 µs ± 3.27 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [112]:
m_fiber_status

array([  0.        , 115.57076529,          inf, ...,          inf,
         0.        ,          inf])

In [23]:
m_fiber_status

array([  0.        , 115.57076529,          inf, ...,          inf,
         0.        ,          inf])

In [71]:
newloc

array([[ 82,  52,  13, ...,   9,  61,   2],
       [ 69,  76, 158, ..., 178, 213, 338]], dtype=int16)

In [40]:
newloc

array([[ 12,  29,  88, ...,  86,  21,  72],
       [ 67, 339,  35, ..., 221,  27, 212]], dtype=int16)

In [24]:
neighbors_flat_flat

array([[    1,     1,   363, ...,   364,     2,   364],
       [    0,     0,     0, ...,     2,     2,     2],
       [    0,     3,     0, ...,     1,     4,     4],
       ...,
       [33299, 33302, 33299, ..., 33662, 33665, 33665],
       [    0,     0,     0, ...,     0,     0,     0],
       [33302, 33302, 33302, ..., 33663, 33663, 33663]], dtype=uint16)

In [68]:
tuple(location)

(array([12, 30, 87, ..., 86, 22, 71], dtype=int16),
 array([ 66, 341,  36, ..., 222,  28, 213], dtype=int16))

In [28]:
edge_lookup = partial(np.ravel_multi_index, dims=(e.macro_params.rows, e.macro_params.full_row))

In [29]:
edge_lookup((3, 5))

1091

In [13]:
type(locationx)

cupy.ndarray